In [26]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import RFE
from sklearn.ensemble import VotingRegressor
from skopt import BayesSearchCV




In [27]:
# Load the data
forest_fires_df = pd.read_csv('forestfires.csv')

# Handle missing values (if any)
forest_fires_df = forest_fires_df.dropna()

In [28]:
%%!
# Convert month to season
def month_to_season(month):
    if month in ['dec', 'jan', 'feb']:
        return 'winter'
    elif month in ['mar', 'apr', 'may']:
        return 'spring'
    elif month in ['jun', 'jul', 'aug']:
        return 'summer'
    else:
        return 'fall'

forest_fires_df['season'] = forest_fires_df['month'].apply(month_to_season)
forest_fires_df = forest_fires_df.drop(columns=['month']) 

# Convert categorical features to numerical using one-hot encoding
forest_fires_df = pd.get_dummies(forest_fires_df, columns=['season'], drop_first=True)

["'#' is not recognized as an internal or external command,",
 'operable program or batch file.']

In [29]:
# Log transform the 'area' to handle skewness
forest_fires_df['log_area'] = np.log1p(forest_fires_df['area'])

# Drop unneccesary columns
forest_fires_df = forest_fires_df.drop(columns=['day', 'area', 'rain', 'X', 'Y', 'month'])


In [30]:
# Split the data into features and target
X = forest_fires_df.drop(['log_area'], axis=1)
y = forest_fires_df['log_area']

In [31]:
# Identify continuous features for standardization
continuous_features = ['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind']

In [32]:
# Create the column transformer with standard scaler
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_features)
    ],
    remainder='passthrough'  # leave the rest of the columns unchanged
)

In [33]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Define the parameter space for Bayesian Optimization
param_space_xgb = {
    'n_estimators': (50, 200),
    'max_depth': (2, 10),
    'learning_rate': (0.01, 0.2, 'log-uniform'),
    'subsample': (0.6, 1.0, 'uniform'),
    'colsample_bytree': (0.6, 1.0, 'uniform')
}

# Bayesian Optimization for XGBoost
bayes_search_xgb = BayesSearchCV(
    estimator=XGBRegressor(random_state=42),
    search_spaces=param_space_xgb,
    n_iter=50,
    cv=3,
    n_jobs=-1,
    verbose=2,
    random_state=42
)

bayes_search_xgb.fit(X_train, y_train)
best_xgb_model_bayes = bayes_search_xgb.best_estimator_

# Evaluate the optimized XGBoost model
xgb_pred_bayes = best_xgb_model_bayes.predict(X_test)

# Evaluate the optimized model
test_rmse_bayes = np.sqrt(mean_squared_error(y_test, xgb_pred_bayes))
test_mae_bayes = mean_absolute_error(y_test, xgb_pred_bayes)
test_r2_bayes = r2_score(y_test, xgb_pred_bayes)

print(f'Test RMSE (XGBoost with Bayesian Optimization): {test_rmse_bayes}')
print(f'Test MAE (XGBoost with Bayesian Optimization): {test_mae_bayes}')
print(f'Test R² (XGBoost with Bayesian Optimization): {test_r2_bayes}')


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

In [35]:
# # Evaluate Feature Importance
# feature_importances = best_model['model'].feature_importances_
# features = X.columns

# # Create a DataFrame for feature importance
# importance_df = pd.DataFrame({
#     'Feature': features,
#     'Importance': feature_importances
# }).sort_values(by='Importance', ascending=False)

# # Plot the feature importance
# plt.figure(figsize=(14, 8))
# sns.barplot(x='Importance', y='Feature', data=importance_df)
# plt.title('Feature Importance for Predicting Fire Size')
# plt.show()

In [36]:
# # Create a pipeline with the preprocessor and CatBoost model
# cat_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('model', CatBoostRegressor(random_state=42, silent=True))
# ])

# # Define the parameter grid for CatBoost
# cat_param_grid = {
#     'model__iterations': [100, 200, 300],
#     'model__depth': [3, 5, 7],
#     'model__learning_rate': [0.01, 0.05, 0.1],
#     'model__l2_leaf_reg': [1, 3, 5]
# }

# # Initialize GridSearchCV for CatBoost
# cat_grid_search = GridSearchCV(estimator=cat_pipeline, param_grid=cat_param_grid, cv=3, n_jobs=-1, verbose=2)

# # Fit the model with GridSearchCV
# cat_grid_search.fit(X_train, y_train)

# # Get the best CatBoost model parameters
# cat_best_params = {key.replace('model__', ''): value for key, value in cat_grid_search.best_params_.items()}

In [37]:
# # Define base models
# base_models = [
#     ('xgb', XGBRegressor(**xgb_best_params, random_state=42)),
#     ('cat', CatBoostRegressor(**cat_best_params, random_state=42, silent=True))
# ]

In [38]:
# # Define stacking model with ElasticNet as the final estimator
# stacking_model_en = StackingRegressor(
#     estimators=base_models,
#     final_estimator=ElasticNet(random_state=42)
# )

# stacking_pipeline_en = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
#     ('model', stacking_model_en)
# ])

# # Fit the stacking model
# stacking_pipeline_en.fit(X_train_poly, y_train)

# # Evaluate Model Performance on Test Data
# y_test_pred_stack_en = stacking_pipeline_en.predict(X_test_poly)
# test_rmse_stack_en = np.sqrt(mean_squared_error(y_test, y_test_pred_stack_en))
# test_mae_stack_en = mean_absolute_error(y_test, y_test_pred_stack_en)
# test_r2_stack_en = r2_score(y_test, y_test_pred_stack_en)

# print(f'Test RMSE (Stacking with ElasticNet): {test_rmse_stack_en}')
# print(f'Test MAE (Stacking with ElasticNet): {test_mae_stack_en}')
# print(f'Test R² (Stacking with ElasticNet): {test_r2_stack_en}')

In [39]:
# # Define stacking model with Gradient Boosting as the final estimator
# stacking_model_gb = StackingRegressor(
#     estimators=base_models,
#     final_estimator=GradientBoostingRegressor(random_state=42)
# )

# stacking_pipeline_gb = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
#     ('model', stacking_model_gb)
# ])

# # Fit the stacking model
# stacking_pipeline_gb.fit(X_train_poly, y_train)

# # Evaluate Model Performance on Test Data
# y_test_pred_stack_gb = stacking_pipeline_gb.predict(X_test_poly)
# test_rmse_stack_gb = np.sqrt(mean_squared_error(y_test, y_test_pred_stack_gb))
# test_mae_stack_gb = mean_absolute_error(y_test, y_test_pred_stack_gb)
# test_r2_stack_gb = r2_score(y_test, y_test_pred_stack_gb)

# print(f'Test RMSE (Stacking with Gradient Boosting): {test_rmse_stack_gb}')
# print(f'Test MAE (Stacking with Gradient Boosting): {test_mae_stack_gb}')
# print(f'Test R² (Stacking with Gradient Boosting): {test_r2_stack_gb}')

In [40]:
# # Fit the best models individually
# best_xgb_model.fit(X_train, y_train)
# best_cat_model.fit(X_train, y_train)

# # Get predictions
# xgb_pred = best_xgb_model.predict(X_test)
# cat_pred = best_cat_model.predict(X_test)

# # Weighted averaging
# weights = [0.6, 0.4]  # Example weights, you can optimize these
# combined_pred = (weights[0] * xgb_pred) + (weights[1] * cat_pred)

# # Evaluate the combined model
# test_rmse_combined = np.sqrt(mean_squared_error(y_test, combined_pred))
# test_mae_combined = mean_absolute_error(y_test, combined_pred)
# test_r2_combined = r2_score(y_test, combined_pred)

# print(f'Test RMSE (Weighted Averaging): {test_rmse_combined}')
# print(f'Test MAE (Weighted Averaging): {test_mae_combined}')
# print(f'Test R² (Weighted Averaging): {test_r2_combined}')


In [41]:
# # Perform Recursive Feature Elimination (RFE) with a Linear Regression model
# rfe = RFE(estimator=LinearRegression(), n_features_to_select=10)
# rfe.fit(X_train, y_train)

# # Select the best features
# X_train_rfe = X_train.loc[:, rfe.support_]
# X_test_rfe = X_test.loc[:, rfe.support_]

# # Train the models again with selected features
# best_xgb_model_rfe = XGBRegressor(**xgb_best_params, random_state=42)
# best_cat_model_rfe = CatBoostRegressor(**cat_best_params, random_state=42, silent=True)

# best_xgb_model_rfe.fit(X_train_rfe, y_train)
# best_cat_model_rfe.fit(X_train_rfe, y_train)

# # Get predictions
# xgb_pred_rfe = best_xgb_model_rfe.predict(X_test_rfe)
# cat_pred_rfe = best_cat_model_rfe.predict(X_test_rfe)

# # Weighted averaging
# weights = [0.6, 0.4]  # Example weights, you can optimize these
# combined_pred_rfe = (weights[0] * xgb_pred_rfe) + (weights[1] * cat_pred_rfe)

# # Evaluate the combined model
# test_rmse_combined_rfe = np.sqrt(mean_squared_error(y_test, combined_pred_rfe))
# test_mae_combined_rfe = mean_absolute_error(y_test, combined_pred_rfe)
# test_r2_combined_rfe = r2_score(y_test, combined_pred_rfe)

# print(f'Test RMSE (Weighted Averaging with RFE): {test_rmse_combined_rfe}')
# print(f'Test MAE (Weighted Averaging with RFE): {test_mae_combined_rfe}')
# print(f'Test R² (Weighted Averaging with RFE): {test_r2_combined_rfe}')

In [42]:
# # Voting Regressor
# voting_regressor = VotingRegressor(estimators=[
#     ('xgb', best_xgb_model_rfe),
#     ('cat', best_cat_model_rfe)
# ])

# # Fit the Voting Regressor
# voting_regressor.fit(X_train_rfe, y_train)

# # Get predictions
# voting_pred = voting_regressor.predict(X_test_rfe)

# # Evaluate the Voting Regressor
# test_rmse_voting = np.sqrt(mean_squared_error(y_test, voting_pred))
# test_mae_voting = mean_absolute_error(y_test, voting_pred)
# test_r2_voting = r2_score(y_test, voting_pred)

# print(f'Test RMSE (Voting Regressor): {test_rmse_voting}')
# print(f'Test MAE (Voting Regressor): {test_mae_voting}')
# print(f'Test R² (Voting Regressor): {test_r2_voting}')

In [43]:
# # Define the parameter space for Bayesian Optimization
# param_space_xgb = {
#     'n_estimators': (50, 200),
#     'max_depth': (2, 10),
#     'learning_rate': (0.01, 0.2, 'log-uniform'),
#     'subsample': (0.6, 1.0, 'uniform'),
#     'colsample_bytree': (0.6, 1.0, 'uniform')
# }

# param_space_cat = {
#     'iterations': (100, 300),
#     'depth': (3, 7),
#     'learning_rate': (0.01, 0.1, 'log-uniform'),
#     'l2_leaf_reg': (1, 5)
# }

# # Bayesian Optimization for XGBoost
# bayes_search_xgb = BayesSearchCV(
#     estimator=XGBRegressor(random_state=42),
#     search_spaces=param_space_xgb,
#     n_iter=50,
#     cv=3,
#     n_jobs=-1,
#     verbose=2,
#     random_state=42
# )

# bayes_search_xgb.fit(X_train, y_train)
# best_xgb_model_bayes = bayes_search_xgb.best_estimator_

# # Bayesian Optimization for CatBoost
# bayes_search_cat = BayesSearchCV(
#     estimator=CatBoostRegressor(random_state=42, silent=True),
#     search_spaces=param_space_cat,
#     n_iter=50,
#     cv=3,
#     n_jobs=-1,
#     verbose=2,
#     random_state=42
# )

# bayes_search_cat.fit(X_train, y_train)
# best_cat_model_bayes = bayes_search_cat.best_estimator_

# # Evaluate the optimized models
# xgb_pred_bayes = best_xgb_model_bayes.predict(X_test)
# cat_pred_bayes = best_cat_model_bayes.predict(X_test)

# # Weighted averaging with Bayesian optimized models
# weights_bayes = [0.6, 0.4]  # Example weights, can be optimized further
# combined_pred_bayes = (weights_bayes[0] * xgb_pred_bayes) + (weights_bayes[1] * cat_pred_bayes)

# # Evaluate the combined model with Bayesian optimization
# test_rmse_combined_bayes = np.sqrt(mean_squared_error(y_test, combined_pred_bayes))
# test_mae_combined_bayes = mean_absolute_error(y_test, combined_pred_bayes)
# test_r2_combined_bayes = r2_score(y_test, combined_pred_bayes)

# print(f'Test RMSE (Weighted Averaging with Bayesian Optimization): {test_rmse_combined_bayes}')
# print(f'Test MAE (Weighted Averaging with Bayesian Optimization): {test_mae_combined_bayes}')
# print(f'Test R² (Weighted Averaging with Bayesian Optimization): {test_r2_combined_bayes}')

In [44]:
# # Define the parameter grid for hyperparameter tuning
# param_grid = {
#     'model__n_estimators': [300, 500, 800],
#     'model__max_depth': [None, 5, 10, 15],
#     'model__min_samples_split': [10, 15, 18],
#     'model__min_samples_leaf': [4, 6, 8]
# }

# # Initialize GridSearchCV
# grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

# # Fit the model with GridSearchCV
# grid_search.fit(X_train, y_train)

# # Get the best model
# best_model = grid_search.best_estimator_
# # Best parameters
# print(f'Best Parameters: {grid_search.best_params_}')

# # Evaluate Model Performance on Training Data
# y_train_pred = best_model.predict(X_train)
# train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
# train_mae = mean_absolute_error(y_train, y_train_pred)
# train_r2 = r2_score(y_train, y_train_pred)

# print(f'Training RMSE: {train_rmse}')
# print(f'Training MAE: {train_mae}')
# print(f'Training R²: {train_r2}')

# # Evaluate Model Performance on Test Data
# y_test_pred = best_model.predict(X_test)
# test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
# test_mae = mean_absolute_error(y_test, y_test_pred)
# test_r2 = r2_score(y_test, y_test_pred)

# print(f'Test RMSE: {test_rmse}')
# print(f'Test MAE: {test_mae}')
# print(f'Test R²: {test_r2}')

# # Cross-Validation
# cv_rmse_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='neg_root_mean_squared_error')
# cv_rmse = -cv_rmse_scores.mean()
# print(f'Cross-validated RMSE: {cv_rmse}')

In [45]:
# # Define the parameter distribution for randomized search
# param_dist = {
#     'model__n_estimators': randint(100, 1000),
#     'model__max_depth': randint(3, 30),
#     'model__min_samples_split': randint(2, 20),
#     'model__min_samples_leaf': randint(1, 20)
# }

# # Initialize RandomizedSearchCV
# random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist, n_iter=50, cv=3, n_jobs=-1, verbose=2, random_state=42)

# # Fit the model with RandomizedSearchCV
# random_search.fit(X_train, y_train)

# # Get the best model
# best_model_random = random_search.best_estimator_

# # Evaluate Model Performance on Test Data
# y_test_pred_random = best_model_random.predict(X_test)
# test_rmse_random = np.sqrt(mean_squared_error(y_test, y_test_pred_random))
# test_mae_random = mean_absolute_error(y_test, y_test_pred_random)
# test_r2_random = r2_score(y_test, y_test_pred_random)

# print(f'Test RMSE (RandomizedSearchCV): {test_rmse_random}')
# print(f'Test MAE (RandomizedSearchCV): {test_mae_random}')
# print(f'Test R² (RandomizedSearchCV): {test_r2_random}')
# print(f'Best Parameters (RandomizedSearchCV): {random_search.best_params_}')